## Pre-processing data

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from shapely.geometry import Point
import matplotlib.pyplot as plt
import fhv

### Prepare Rasters
Here, we clip global-scale rasters to Peruvian national boundary.
1. DEM from HydroSHEDS
2. 10-year flood inundation from GLOFRIS.
3. Population from LandScan
4. LandCover data from [European Space Agency (ESA)'s Climate Change Initiative (CCI)](https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-land-cover?tab=overview)
5. DistID (District ID of the district shapefile is converted to Raster)

In [18]:
# Peruvian national boundary
shp_fn = './data/per_admbnda_adm0_2018.shp'

# 1. Hydroshed DEM
# - Crop raster with Peruvian national boundary
rst_fn = '/Users/dlee/data/gis/hydrosheds/dem_void/sa_dem_30s/sa_dem_30s'
out_fn = './data/aligned/raw/dem30s_peru.tif'
fhv.CropRasterShape(rst_fn, shp_fn, out_fn, all_touched=False)
# - Reproject to UTM zone 17S (EPSG:32717)
outpath = './data/aligned/raw/dem30s_peru_projected.tif'
fhv.ReprojectRaster(out_fn, outpath, new_crs='EPSG:32717')

# 2. GLOFRIS 10-year inundation
rst_fn = '/Users/dlee/data/inundation/glofris/inun_dynRout_RP_00010.tif'
out_fn = './data/aligned/raw/inundation_00010.tif'
fhv.CropRasterShape(rst_fn, shp_fn, out_fn, all_touched=False)

# 3. LandScan population
rst_fn = '/Users/dlee/data/population/landscan/LandScan Global 2017/lspop2017'
out_fn = './data/aligned/raw/popu_admin_landscan17.tif'
fhv.CropRasterShape(rst_fn, shp_fn, out_fn, all_touched=False)
# # Total population of Peru in 2017 was 31,237,385 (INEI) or 32,165,485 (UNPD)
# with rasterio.open(out_fn) as src: 
#     popu = src.read().squeeze()
#     popu17 = np.sum(popu[popu != popu[0,0]])    # 30,931,229

# 4. LandCover
# Global LandCover layer is manually clipped using QGIS to the Peruvian national boundary
source = '/Users/dlee/data/landcover/C3S-LC-L4-LCCS-Map-300m-P1Y-2018-v2.1.1.nc'
out_fn = './data/lcss_aligned.tif'

# 5. DistID (converting administrative boundary polygon to raster)
shp_fn = './data/DISTRITOS.shp'
rst_fn = './data/aligned/raw/dem30s_peru.tif'
out_fn = './data/aligned/raw/distid_30s.tif'
# Open the shapefile with GeoPandas
dist = gpd.read_file(shp_fn)
# Open the raster file as a template for feature burning using rasterio
rst = rasterio.open(rst_fn)
# Copy and update the metadata frm the input raster for the output
meta = rst.meta.copy()
meta.update(dtype=rasterio.int32)
# Before burning it, we need to 
dist = dist.assign(IDDIST_int = dist.IDDIST.values.astype(rasterio.int32))
# Burn the features into the raster and write it out
with rasterio.open(out_fn, 'w+', **meta) as out:
    out_arr = out.read(1)
    shapes = ((geom, value) for geom, value in zip(dist.geometry, dist.IDDIST_int))
    burned = rasterio.features.rasterize(shapes=shapes, fill=0, out=out_arr, 
                                         transform=out.transform,
                                         all_touched=False)
    out.write_band(1, burned)
    print('%s is saved' % out_fn)

./data/aligned/raw/dem30s_peru.tif is saved.
./data/aligned/raw/dem30s_peru_projected.tif is saved.
./data/aligned/raw/inundation_00010.tif is saved.
./data/aligned/raw/popu_admin_landscan17.tif is saved.
./data/aligned/raw/distid_30s.tif is saved


### Alignment of Rasters
Then, we align rasters in order to match their extent and resolution. This is done manually by QGIS > Raster > Align Rasters. The raster data includes:
1. DEM: './data/alinged/raw/dem30_peru_Projected.tif' (reference raster; UTM zone 17S (EPSG:32717))
2. Inundation: './data/aligned/raw/inundation_00010.tif'
3. population: './data/aligned/raw/popu_admin_landscan17.tif'
4. LandCover: './data/aligned/raw/landcover_peru.tif'
5. DistID: './data/aligned/raw/distid_30s.tif'

### Health facilities
We obtained the information of health facilities (location, name, level, etc.) from [GeoMINSA](http://www.geominsa.minsa.gob.pe:8080/geominsa/) as Excel format. Here, we read the data and create a shapefile of health facilities.

In [2]:
# Load health facilities obtained from GeoMINSA
filn_in = os.path.join('./data/health_facility_MINSA.xls')
df = pd.read_excel(filn_in, header=0).dropna()
df = df.rename(columns={'Distrito':'district',
                       'Latitud':'x',
                       'Longitud':'y',
                       'Categoría':'category'})
df.head()
# GeoDataFrame needs a shapely object
df['Coordinates'] = list(zip(df.x, df.y))           # Coordinates
df['Coordinates'] = df['Coordinates'].apply(Point)  # tuples to Shapely's Point
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(df, crs=crs, geometry='Coordinates')
# Select specific types of facilities
gdf = gdf[gdf.category.isin(['I-4','II-1','II-2','II-E','III-1','III-2','III-E'])]
# Write ESRI shapefile
if True:
    filn_out = os.path.join('data/health_facility_MINSA.shp')
    gdf.to_file(filn_out, encoding='utf-8')
    print('%s is saved.' % filn_out)

data/health_facility_MINSA.shp is saved.


./data/lcss_class_peru_projected.tif is saved.


### TODO
- polygonize inundation (>1.0m)
- calculate accessibility2
- function and script organize

./data/dem_30s_peru_projected.tif is saved.


### Flood Inundation Barriers
We polygonize the cells with flood depth over 1.0m.




In [31]:
# Read/Modify/Save the inundation raster
in_ras = './data/aligned/inundation_peru.tif'
out_ras = './data/aligned/polygonized/inundation_peru_1m.tif'
with rasterio.open(in_ras, 'r') as src:
    meta = src.meta.copy()
    # Ignore cells with flood depth under 1.0 meter.
    inun = src.read(1)
    inun[inun < 10] = -32768
    with rasterio.open(out_ras, 'w', **meta) as dest:
        dest.write_band(1, inun)
        print('%s is saved.' % out_ras)

# inun = rasterio.open() 
# inun = inun.read(1)
np.unique(inun)

./data/aligned/polygonized/inundation_peru_1m.tif is saved.


array([-32768,     10,     11,     12,     13,     14,     15,     16,
           17,     18,     19,     20,     21,     22,     23,     24,
           25,     26,     27,     28,     29,     30,     31,     32,
           33,     34,     35,     36,     37,     38,     39,     40,
           41,     42,     43,     44,     45,     46,     47,     48,
           49,     50,     51,     52,     53,     54,     55,     56,
           57,     58,     59,     60,     61,     62,     63,     64,
           65,     66,     67,     68,     69,     70,     71,     72,
           73,     74,     75,     76,     77,     78,     79,     80,
           81,     82,     83,     84,     85,     86,     87,     88,
           89,     90,     91,     92,     93,     94,     95,     96,
           97,     98,     99,    100,    101,    102,    103,    104,
          105,    106,    107,    108,    111,    113,    123,    127,
          129,    132,    137,    138,    147,    152,    156,    162,
      

In [27]:
inun.shape

(2238, 1537)

### Roads
Roads data is obtained from [HOTOSM (OpenStreetMap) from Humnitarian Data Exchange](https://data.humdata.org/dataset/hotosm_per_roads).

In [49]:
filn_in = '/Users/dlee/data/per/roads/hotosm_per_roads_lines_shp/hotosm_per_roads_lines.shp'
road = gpd.read_file(filn_in)
# Select Primary, Secondary, and Tertiary roads
road = road.loc[road['highway'].isin(['primary', 'secondary', 'tertiary'])]
# Type and Class
road = road.drop(['osm_id','name','surface','smoothness','width','lanes','oneway','bridge','layer','z_index'], axis=1)
road_class = {1:'primary',2:'secondary',3:'tertiary'}
road['class'] = 1
road.loc[road.highway == 'secondary', 'class'] = 2
road.loc[road.highway == 'tertiary', 'class'] = 3
# Save
if True:
    filn_out = './data/road_osm.shp'
    road.to_file(filn_out)
    print('%s is saved.' % filn_out)

./data/road_osm.shp is saved.


### Education facilities
Education facilities is obtained from [Ministry of Education > ESCALE](http://escale.minedu.gob.pe/mapas).
https://wenr.wes.org/2015/04/education-in-peru

### Rivers and Wetlands
The hydrography data of Peru is obtained from [Humanitarian Data Exchange (HDX) (Instituto Geográfico Nacional - IGN)](https://data.humdata.org/dataset/hidrografia-de-peru).


In [ ]:
# - Automatic code for LandCover
# from netCDF4 import Dataset
# import rioxarray
# import xarray
# filn = '/Users/dlee/data/landcover/C3S-LC-L4-LCCS-Map-300m-P1Y-2018-v2.1.1.nc'
# nc_fid = Dataset(filn, 'r')
# # Use RioXarray
# xds = xarray.open_dataset(filn)
# xds.rio.set_crs("epsg:4326")
# xds["lccs_class"].rio.to_raster('.\test.tif')